In [16]:
import pyaudio
import wave
import os

if 'output.wav' in os.listdir():
    os.remove('output.wav')
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "output.wav"

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("* recording")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

* recording
* done recording


In [18]:
import pygame
file = 'output.wav'
pygame.init()
pygame.mixer.init()
pygame.mixer.music.load(file)
pygame.mixer.music.play()

## model

In [19]:
import tensorflow as tf
import librosa
import os
import numpy as np

In [20]:
TFLite_model = 'lite-model_ASR_TFLite_pre_trained_models_English_1.tflite'

In [21]:
audio = 'output.wav'

In [22]:
signal, _ = librosa.load(os.path.expanduser(audio), sr=16000, mono=True)

In [23]:
interpreter = tf.lite.Interpreter(model_path=TFLite_model)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [24]:
interpreter.resize_tensor_input(input_details[0]["index"], signal.shape)
interpreter.allocate_tensors()
interpreter.set_tensor(input_details[0]["index"], signal)
interpreter.set_tensor(
    input_details[1]["index"],
    np.array(0).astype('int32')
)
interpreter.set_tensor(
    input_details[2]["index"],
    np.zeros([1,2,1,320]).astype('float32')
)
interpreter.invoke()
hyp = interpreter.get_tensor(output_details[0]["index"])

print("".join([chr(u) for u in hyp]))

    said    ind ig  na  nt  mar in  ue  d   h   
